In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import time
from string import punctuation

In [ ]:
'''
Data Cleaning for the purposes of random selecting, but model should learn all the symbols that are used on human-made poems
text = open('/content/шын гашыкпын.txt', 'rb').read().decode(encoding='utf-8')
import re

text = text.translate(str.maketrans("", "", punctuation))

text = text.lower()
text = re.sub(r'[.,0145«»"\'-?:!;–—…﻿tacehopty]', '', text)
'''

'\ntext = open(\'/content/шын гашыкпын.txt\', \'rb\').read().decode(encoding=\'utf-8\')\nimport re\n \ntext = text.translate(str.maketrans("", "", punctuation))\n\ntext = text.lower()\ntext = re.sub(r\'[.,0145«»"\'-?:!;–—…\ufefftacehopty]\', \'\', text)\n'

In [ ]:
text = open('/content/Full_Dataset.txt', 'rb').read().decode(encoding='utf-8')

In [ ]:
text[0:5000]

'\ufeffБай болған дəулеті мол Тілеуберді,\r\nБайлықпенен меңгерген тамам елді.\r\nТөрт түлік мал өргенде құрттай қайнап,\r\nҚұмырсқадай қыбырлап жапқан жерді.\r\nТор-қасқалы, қазмойын жылқыларын\r\nЕкі-үш мыңдап он шақты қосқа бөлді.\r\nТүйелері түздерде бота тауып,\r\nҚойы жүрген жерінен қозы терді.\r\nЖас кезінде қыз таңдап кезіп елді,\r\nӨз бойына лайықты іздеп теңді,\r\nМың жылқыны біржола матап беріп,\r\nАлтынай дейтін сұлуды алып еді.\r\nТоқсан нарға жүк артып безеп, жасап,\r\nАлтынайға əкесі құл, күң берді.\r\nШұнақ деген жігіт пен Тезек атты\r\nБір жас қыз, құл, күңдермен ере келді.\r\nШұнақ пенен Тезек те қосты көңіл,\r\nҚұлдық, күндік болса да көрген өмір,\r\nӨздерінше қызықты дəурен сүріп,\r\nТəтті өмір тəрізді болды о бір.\r\nШөккен түйе, көгенді қозы арасы, –\r\nЖата-тұғын мекені күл мен көмір.\r\nСөйтіп Шұнақ Тезекпен осы өмірде\r\nТірі жанға сездірмей жүрді үш-төрт жыл.\r\nСодан кейін құрсақты болып Тезек,\r\nТорсық шеге, ақ сазан тапты бір ұл.\r\nҰл туды деп шашулар шаш

In [ ]:
# The unique characters in the file
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

121 unique characters


In [ ]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [ ]:
# The maximum length sentence you want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [ ]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [ ]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [ ]:
# Batch size
BATCH_SIZE = 10

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<_BatchDataset element_spec=(TensorSpec(shape=(10, 100), dtype=tf.int64, name=None), TensorSpec(shape=(10, 100), dtype=tf.int64, name=None))>

In [ ]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [ ]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(10, 100, 121) # (batch_size, sequence_length, vocab_size)


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (10, None, 256)           30976     
                                                                 
 gru (GRU)                   (10, None, 1024)          3938304   
                                                                 
 dense (Dense)               (10, None, 121)           124025    
                                                                 
Total params: 4,093,305
Trainable params: 4,093,305
Non-trainable params: 0
_________________________________________________________________


In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [ ]:
sampled_indices

array([ 50,  14,  91,  68,  29, 113,  65,  27,   5, 119, 119,  65,  21,
        31,  99,  87,  63, 102,  20,  30,  51,   0,  32,  59,  54,  69,
       111,  88,  14,  75,   0,  80,  85, 108,  21,  58,   4,  72,  42,
        41,  79,  81,  14,  84,  11, 109,  85,  81,  21, 104,  86,  71,
        47,  38,  42,  98,  86,  30,  89,  82,  58,  79,  44, 115,  26,
        84,   3,  60,   2, 100,   1,  15, 103,  86, 114,  78,  89,  60,
       101,  84,  18,  41,  87,  47,  82,  87,  49,  41,  77,  77,  22,
        64,  10,  82,  97,   4,  22,  86,  18, 113])

In [ ]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)

In [ ]:
model.compile(optimizer='adam', loss=loss)

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:
EPOCHS = 23

In [ ]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/23
531/531 [==============================] - 21s 33ms/step - loss: 2.2148
Epoch 2/23
531/531 [==============================] - 16s 28ms/step - loss: 1.8554
Epoch 3/23
531/531 [==============================] - 15s 28ms/step - loss: 1.7120
Epoch 4/23
531/531 [==============================] - 16s 27ms/step - loss: 1.6102
Epoch 5/23
531/531 [==============================] - 15s 27ms/step - loss: 1.5266
Epoch 6/23
531/531 [==============================] - 16s 28ms/step - loss: 1.4477
Epoch 7/23
531/531 [==============================] - 16s 28ms/step - loss: 1.3668
Epoch 8/23
531/531 [==============================] - 16s 28ms/step - loss: 1.2823
Epoch 9/23
531/531 [==============================] - 16s 28ms/step - loss: 1.1963
Epoch 10/23
531/531 [==============================] - 16s 28ms/step - loss: 1.1111
Epoch 11/23
531/531 [==============================] - 16s 28ms/step - loss: 1.0355
Epoch 12/23
531/531 [==============================] - 16s 28ms/step - loss: 0.9673
E

In [ ]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_23'

In [ ]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (1, None, 256)            30976     
                                                                 
 gru_1 (GRU)                 (1, None, 1024)           3938304   
                                                                 
 dense_1 (Dense)             (1, None, 121)            124025    
                                                                 
Total params: 4,093,305
Trainable params: 4,093,305
Non-trainable params: 0
_________________________________________________________________


In [ ]:
def generate_text(model, start_string,t):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 300

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = t

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [ ]:
print(generate_text(model, start_string=u'''Жүреді түні бойы жұлдыз санап,
Ғашығын іздеп, көзін көкке қадап,
Ермегі сүмбі-сынды Торытөбел
Оқтаудай ішін тартып қатқан жарап.
''',t=0.000001))

Жүреді түні бойы жұлдыз санап,
Ғашығын іздеп, көзін көкке қадап,
Ермегі сүмбі-сынды Торытөбел
Оқтаудай ішін тартып қатқан жарап.
Нарыға мінген жерден төрт баласына
Оларды қара көңіл боп тұрсын не?.. Абылай да, –
Бұлдыр қақ бірін-бірі серпілгендей.
Байлаулы бала қайнап, амандасып
Құлақ түр боп жүрген жері үшін,
Жүгірген арымақтың деп оңбайлап маңайында?
Көлеңке үй тоқтатқан оңалауға
Төсенген түрілдейді күйші кетсе,
Мық
